In [14]:
# 필요한 패키지 import
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from keras.applications import xception
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [15]:
# 글로벌 변수 선언
FAST_RUN = False
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
IMAGE_SIZE = (IMAGE_WIDTH,IMAGE_HEIGHT)
IMAGE_CHANNELS = 3 # 컬러 사진이므로 채널이 3(rgb)
batch_size = 15

In [16]:
# 종 분류 모델 가져오기
from tensorflow.compat.v2.keras.models import model_from_json

# json 파일 열기
json_file = open('CNN_dog_and_cat_breed_model.json','r')
loaded_model_json = json_file.read()
json_file.close()

# json 파일로부터 model 로드하기
loaded_model = model_from_json(loaded_model_json)

# 로드한 model에 weight 로드하기
loaded_model.load_weights('CNN_dog_and_cat_breed_model.h5')

In [17]:
# 모델 컴파일
loaded_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

# 모델 요약
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0

In [20]:
def final_predict_xception(final_model, img_path, topk = 3):
    
    # obtain predicted vector
    img = load_img(img_path, target_size = (299, 299))
    x = img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    y = xception.preprocess_input(x)/255
    print(x,y)
    predicted_vector = final_model.predict(y)
    
    # return dog breed that is predicted by the model
    results = sorted(enumerate(predicted_vector[0]), reverse=True, key=lambda x:x[1])
    classes_ind = [x[0] for x in results]
    classes = [dog_names[x] for x in classes_ind][:topk]
    probs = [x[1] for x in results][:topk]
    
    # plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
    
    img = mpimg.imread(img_path)
    ax1.imshow(img)
    
    ind = np.arange(len(classes))
    ax2.bar(ind, probs, align='center', alpha = .75)
    ax2.set_xticks(ind)
    ax2.set_xticklabels(classes, rotation=90)

In [21]:
src = './dogvscat/test1/'
file = random.choice(os.listdir(src))
fullpath = src+file

final_predict_xception(loaded_model,fullpath,topk=5)

[[[[-0.3333333  -0.34117645 -0.27843136]
   [-0.34117645 -0.3490196  -0.2862745 ]
   [-0.34117645 -0.3490196  -0.2862745 ]
   ...
   [ 0.5372549   0.5372549   0.5372549 ]
   [ 0.43529415  0.43529415  0.43529415]
   [ 0.38823533  0.38823533  0.38823533]]

  [[-0.3490196  -0.35686272 -0.29411763]
   [-0.32549018 -0.3333333  -0.27058822]
   [-0.30196077 -0.3098039  -0.24705881]
   ...
   [ 0.5764706   0.5764706   0.5764706 ]
   [ 0.47450984  0.47450984  0.47450984]
   [ 0.39607847  0.39607847  0.39607847]]

  [[-0.34117645 -0.3490196  -0.2862745 ]
   [-0.3098039  -0.31764704 -0.25490195]
   [-0.2862745  -0.29411763 -0.23137254]
   ...
   [ 0.45882356  0.45882356  0.45882356]
   [ 0.3803922   0.3803922   0.3803922 ]
   [ 0.30980396  0.30980396  0.30980396]]

  ...

  [[-0.8117647  -0.8117647  -0.79607844]
   [-0.81960785 -0.81960785 -0.8039216 ]
   [-0.8352941  -0.8352941  -0.81960785]
   ...
   [-0.77254903 -0.77254903 -0.77254903]
   [-0.7647059  -0.7647059  -0.7647059 ]
   [-0.7490196  

ValueError: in user code:

    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/team2/anaconda3/lib/python3.7/site-packages/tensorflow/pyth